In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score,roc_curve,auc
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from  sklearn import svm
from sklearn.tree import DecisionTreeClassifier

In [21]:
data=pd.read_csv('C:\\Users\miao\Desktop\\data.csv',index_col=0,encoding = 'gbk')

In [54]:
print(data.head(3))

     custid      trade_no bank_card_no  low_volume_percent  \
5   2791858  2.018050e+31          卡号1                0.01   
10   534047  2.018050e+31          卡号1                0.02   
12  2849787  2.018050e+31          卡号1                0.04   

    middle_volume_percent  take_amount_in_later_12_month_highest  \
5                    0.99                                      0   
10                   0.94                                   2000   
12                   0.96                                      0   

    trans_amount_increase_rate_lately  trans_activity_month  \
5                                0.90                  0.55   
10                               1.28                  1.00   
12                               1.00                  1.00   

    trans_activity_day  transd_mcc  ...  loans_max_limit  loans_avg_limit  \
5                0.313        17.0  ...           2900.0           1688.0   
10               0.458        19.0  ...           3500.0           1758

In [55]:
#单独提取出y列标签，和其余的88列标记为x
y=data['status']
X=data.drop('status',axis=1)
#X值的行列数，以及y的分布类型
print('X.shape:',X.shape)
print('y的分布:',y.value_counts())

X.shape: (4754, 88)
y的分布: 0    3561
1    1193
Name: status, dtype: int64


In [64]:
data.dtypes
X.dtypes

custid                                     int64
trade_no                                 float64
bank_card_no                              object
low_volume_percent                       float64
middle_volume_percent                    float64
take_amount_in_later_12_month_highest      int64
trans_amount_increase_rate_lately        float64
trans_activity_month                     float64
trans_activity_day                       float64
transd_mcc                               float64
trans_days_interval_filter               float64
trans_days_interval                      float64
regional_mobility                        float64
student_feature                          float64
repayment_capability                       int64
is_high_user                               int64
number_of_trans_from_2011                float64
first_transaction_time                   float64
historical_trans_amount                    int64
historical_trans_day                     float64
rank_trad_1_month   

In [69]:
delete_columns = ["custid", "trade_no", "bank_card_no", "first_transaction_time", "source", 
                  "id_name", "latest_query_time", "loans_latest_time","student_feature"]
X.drop(delete_columns, axis=1)

low_volume_percent  middle_volume_percent  \
5                    0.01                   0.99   
10                   0.02                   0.94   
12                   0.04                   0.96   
13                   0.00                   0.96   
14                   0.01                   0.99   
15                   0.02                   0.98   
16                   0.02                   0.98   
18                   0.02                   0.98   
20                   0.03                   0.65   
26                   0.01                   0.99   
28                   0.01                   0.99   
29                   0.01                   0.99   
30                   0.00                   0.73   
33                   0.01                   0.79   
37                   0.01                   0.89   
41                   0.01                   0.99   
43                   0.02                   0.98   
46                   0.02                   0.98   
48                   0.01                   0.99   
49                   0.01                   0.96   
50                   0.02                   0.98   
52                   0.02                   0.98   
53                   0.00                   0.59   
60                   0.01                   0.99   
61                   0.02                   0.78   
62                   0.40                   0.60   
64                   0.02                   0.96   
71                   0.01                   0.91   
80                   0.01                   0.99   
88                   0.02                   0.64   
...                   ...                    ...   
11923                0.02                   0.98   
11924                0.01                   0.99   
11926                0.00                   1.00   
11927                0.02                   0.98   
11930                0.06                   0.94   
11932                0.17                   0.83   
11933                0.01                   0.99   
11936                0.00                   0.58   
11938                0.11                   0.89   
11940                0.03                   0.97   
11941                0.00                   1.00   
11945                0.01                   0.61   
11947                0.05                   0.95   
11950                0.01                   0.99   
11953                0.02                   0.98   
11957                0.05                   0.62   
11958                0.07                   0.93   
11959                0.00                   0.83   
11964                0.02                   0.98   
11967                0.00                   1.00   
11968                0.01                   0.96   
11969                0.04                   0.96   
11972                0.01                   0.91   
11974                0.02                   0.40   
11977                0.09                   0.91   
11978                0.05                   0.68   
11986                0.01                   0.99   
11988                0.00                   1.00   
11991                0.05                   0.95   
11992                0.01                   0.99   

       take_amount_in_later_12_month_highest  \
5                                          0   
10                                      2000   
12                                         0   
13                                      2000   
14                                         0   
15                                      2000   
16                                         0   
18                                         0   
20                                         0   
26                                       500   
28                                         0   
29                                       500   
30                                      5000   
33                                         0   
37                                      4000   
41            

In [70]:
#选取数值型特征
X_num=X.select_dtypes('number').copy()
print(X_num.shape)
type(X_num.mean())
#使用均值填充缺失值
X_num.fillna(X_num.mean(),inplace=True)
#观察数值型以外的变量
X_str=X.select_dtypes(exclude='number').copy()
X_str.describe()
#把reg_preference用虚拟变量代替，其它三个变量删除
X_str['reg_preference_for_trad'] = X_str['reg_preference_for_trad'].fillna(X_str['reg_preference_for_trad'].mode()[0])
X_str_dummy = pd.get_dummies(X_str['reg_preference_for_trad'])
X_str_dummy.head()
#合并数值型变量和名义型（字符型）变量
X_cl = pd.concat([X_num,X_str_dummy],axis=1,sort=False)
#X_cl.shape
X_cl.head()
"""
#数据标准化和归一化
from sklearn import preprocessing
min_max_scale = preprocessing.MinMaxScaler()
min_max_data = min_max_scale.fit_transform(X_cl)
"""
from sklearn import preprocessing
X_cl = preprocessing.scale(X_cl)


(4754, 82)


d:\programdata\python36\lib\site-packages\ipykernel_launcher.py:25: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by the scale function.
d:\programdata\python36\lib\site-packages\sklearn\preprocessing\data.py:180: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [84]:
#以三七比例分割训练集和测试集
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from  sklearn import svm
from sklearn.tree import DecisionTreeClassifier


random_state = 1118
X_train,X_test,y_train,y_test = train_test_split(X_cl,y,test_size=0.3,random_state=1118)
print(X_train.shape)
print(X_test.shape)


print('建立xgboost模型...')
xgboost_model=XGBClassifier()
xgboost_model.fit(X_train,y_train)
#用建立好的xgboost模型运用到训练集和测试集上，进行预测
y_train_pred = xgboost_model.predict(X_train)
y_test_pred = xgboost_model.predict(X_test)
print('训练集：{:.4f}'.format(roc_auc_score(y_train, y_train_pred)))
print('测试集：{:.4f}'.format(roc_auc_score(y_test, y_test_pred)))


print('建立逻辑回归模型...')
lr = LogisticRegression(C=0.09,random_state=0,penalty='l1')
lr.fit(X_train, y_train)
#用建立好的lr模型运用到训练集和测试集上，进行预测
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)
print('训练集：{:.4f}'.format(roc_auc_score(y_train, y_train_pred)))
print('测试集：{:.4f}'.format(roc_auc_score(y_test, y_test_pred)))

print('svm...')
Lin_SVC = svm.SVC(probability=True)
Lin_SVC.fit(X_train,y_train)
    
y_train_pred = Lin_SVC.predict(X_train)
y_test_pred = Lin_SVC.predict(X_test)
print('训练集：{:.4f}'.format(roc_auc_score(y_train, y_train_pred)))
print('测试集：{:.4f}'.format(roc_auc_score(y_test, y_test_pred)))


(3327, 87)
(1427, 87)
建立xgboost模型...
训练集：0.7386
测试集：0.6329
建立逻辑回归模型...
训练集：0.6545
测试集：0.6145
svm...


d:\programdata\python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\programdata\python36\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


训练集：0.6842
测试集：0.5968
